## Limpieza de Datos:

Elimina o trata los valores nulos, si los hay, en las columnas clave para asegurar que los datos
estén completos.
Verifica la consistencia y corrección de los datos para asegurarte de que los datos se
presenten de forma coherente.
Realiza cualquier ajuste o conversión necesaria en las columnas (por ejemplo, cambiar tipos de
datos) para garantizar la adecuación de los datos para el análisis estadístico.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None) 

In [2]:
df_final = pd.read_csv("../data/df_merged.csv")

In [3]:
df_final.head(5)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Canada,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,Canada,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


Primero, pasamos los nombres de las columnas a minúscula y se sustituye el espacio por un "_".  

In [4]:
def limpiar_columnas(df):
    """
    Convierte los nombres de las columnas a minúsculas
    y reemplaza los espacios por guiones bajos.
    """
    df.columns = (
        df.columns
        .str.lower()
        .str.replace(" ", "_")
    )
    return df

In [5]:
df_final = limpiar_columnas(df_final)

Se crea una columna nueva con clientes activos y con los que han cancelado. Posteriormente, cambiamos el tipo de dato de "cancellation_month" y "cancellationy_ear" a Int64 para mantener los nulos ya que son clientes que no han cancelado su membresía.

In [6]:
df_final["customer_status"] = np.where(df_final["cancellation_year"].isna(), "Active", "Cancelled")

In [7]:
df_final = df_final.astype({"cancellation_year": "Int64", "cancellation_month": "Int64"})

In [8]:
#Se comprueba el porcentaje de clientes activos y cancelados, que coincide con el porcentaje de nulos en la columna "cancellation_year", lo que indica que el proceso de creación de la columna "customer_status" 
# se ha realizado correctamente.
df_final["customer_status"].value_counts(normalize=True) * 100

customer_status
Active       87.703091
Cancelled    12.296909
Name: proportion, dtype: float64

In [9]:
df_final.groupby("customer_status")["loyalty_number"].nunique()

customer_status
Active       14670
Cancelled     2067
Name: loyalty_number, dtype: int64

Pasamos a gestionar los valores negativos de "Salary" antes de gestionar los nulos de esta categoría.

In [10]:
df_final["salary"].isna().sum()

np.int64(102260)

In [11]:
(df_final["salary"] < 0).sum()

np.int64(480)

Comprobamos si de los 480 registros con valores negativos hay algún cliente que tenga datos válidos en otro mes. Si no lo tienen, pasamos estos datos a nulos.

In [12]:
clientes_negativos = df_final.loc[df_final["salary"] < 0, "loyalty_number"].unique()
len(clientes_negativos)

20

In [13]:
df_neg = df_final[df_final["loyalty_number"].isin(clientes_negativos)]
df_neg.groupby("loyalty_number")["salary"].count()

loyalty_number
115505    24
152016    24
194065    24
212128    24
232755    24
239955    24
347013    24
364596    24
366599    24
430398    24
436087    24
491242    24
525245    24
542976    24
603070    24
729561    24
734647    24
790475    24
906428    24
959977    24
Name: salary, dtype: int64

In [14]:
#Se decide eliminar los valores negativos de la columna "salary" y reemplazarlos por NaN, ya que no es posible tener un salario negativo y estos valores podrían afectar negativamente a los análisis posteriores.
df_final.loc[df_final["salary"] < 0, "salary"] = np.nan

In [15]:
#Comprobación de que ya no hay negativos y cómo ha afectado esto al porcentaje de nulos en esta columna. 
(df_final["salary"] < 0).sum()

np.int64(0)

In [16]:
df_final["salary"].isna().sum() * 100 / len(df_final)

np.float64(25.44580939171785)

In [17]:
#Se comprueba la distribución de la columna salary para comparar antes y después de gestionar los valores nulos en la siguiente celda. 
df_final["salary"].describe()

count    301020.000000
mean      79441.628829
std       34704.340158
min       15609.000000
25%       59278.000000
50%       73523.000000
75%       88626.000000
max      407228.000000
Name: salary, dtype: float64

In [18]:
#Se decide imputar los valores nulos de la columna "salary" utilizando la mediana de cada grupo definido por las columnas "education" y "loyalty_card", ya que estas variables podrían estar 
# relacionadas con el salario y la mediana es una medida robusta frente a valores atípicos.
def imputar_salary(df, columna="salary"):
    """
    Imputa valores nulos de salary de forma jerárquica:
    1) Mediana por (education, loyalty_card)
    2) Mediana por education
    3) Mediana global
    """
    
    df = df.copy()
    
    #Mediana por education + loyalty_card
    mediana_grupo = df.groupby(["education", "loyalty_card"])[columna].transform("median")
    df[columna] = df[columna].fillna(mediana_grupo)
    
    #Mediana por education
    mediana_edu = df.groupby("education")[columna].transform("median")
    df[columna] = df[columna].fillna(mediana_edu)
    
    #Mediana global
    mediana_global = df[columna].median()
    df[columna] = df[columna].fillna(mediana_global)
    
    return df

In [19]:
df_final = imputar_salary(df_final)

In [20]:
# Aquí se observa que tras imputar los valores nulos, la mediana y la media son bastante similares a la comprobación anterior, lo que indica que la imputación no ha sesgado significativamente la distribución de los salarios.
df_final["salary"].describe()

count    403760.000000
mean      77922.166857
std       30078.197775
min       15609.000000
25%       64001.000000
50%       73479.000000
75%       82940.000000
max      407228.000000
Name: salary, dtype: float64

In [21]:
df_final["salary"].isna().sum()

np.int64(0)

Para la gestión de los valores nulos en la variable Salary se evaluaron distintas estrategias de imputación. Se probó un método basado en KNN, observándose que, aunque la media se mantenía estable, la desviación estándar disminuía notablemente y la distribución perdía parte de su asimetría original. Esto indicaba una homogeneización artificial de la variable y una reducción de su variabilidad real.

Se consideró más apropiado utilizar una imputación basada en la mediana dentro de grupos homogéneos definidos por `education` y `loyalty_card`. Esta estrategia:

Mantiene la estructura y dispersión original de la variable.

Respeta la asimetría observada en la distribución salarial.

Utiliza criterios conceptualmente coherentes con el nivel de ingresos (formación y tipo de tarjeta).

Resulta más interpretable y fácilmente defendible en un análisis estadístico descriptivo.

Por estos motivos, se optó por imputar los valores nulos de Salary utilizando la mediana por grupo de Education y Loyalty Card.

In [22]:
# Se comprueba que tras la limpieza los tipos de datos son correctos y que no hay valores nulos en la columna "salary", lo que indica que el proceso de limpieza se ha realizado correctamente.
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403760 entries, 0 to 403759
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               403760 non-null  int64  
 1   year                         403760 non-null  int64  
 2   month                        403760 non-null  int64  
 3   flights_booked               403760 non-null  int64  
 4   flights_with_companions      403760 non-null  int64  
 5   total_flights                403760 non-null  int64  
 6   distance                     403760 non-null  int64  
 7   points_accumulated           403760 non-null  float64
 8   points_redeemed              403760 non-null  int64  
 9   dollar_cost_points_redeemed  403760 non-null  int64  
 10  country                      403760 non-null  object 
 11  province                     403760 non-null  object 
 12  city                         403760 non-null  object 
 13 

In [23]:
df_final.isna().sum()

loyalty_number                      0
year                                0
month                               0
flights_booked                      0
flights_with_companions             0
total_flights                       0
distance                            0
points_accumulated                  0
points_redeemed                     0
dollar_cost_points_redeemed         0
country                             0
province                            0
city                                0
postal_code                         0
gender                              0
education                           0
salary                              0
marital_status                      0
loyalty_card                        0
clv                                 0
enrollment_type                     0
enrollment_year                     0
enrollment_month                    0
cancellation_year              354110
cancellation_month             354110
customer_status                     0
dtype: int64

In [25]:
df_final.to_csv("../data/df_final.csv", index=False)